<a href="https://colab.research.google.com/github/aaalexlit/tf-advanced-techniques-spec/blob/main/course_1_custom_models/Week1_Custom_loss_as_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Huber Loss hyperparameter and class

1. Include hyperparameters in defining loss functions. 
1. Implement a custom loss as an object by inheriting the [Loss](https://www.tensorflow.org/api_docs/python/tf/keras/losses/Loss) class.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras import Sequential
from tensorflow.keras.losses import Loss

## Dataset

`xs` and `ys` where the relationship is $y = 2x-1$.  
prediction for `x=10` should be close to `19`.

In [2]:
# inputs
xs = np.array([-1.0,  0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)

# labels
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

## Custom loss with hyperparameter

The `loss` argument in `model.compile()` only accepts functions that accepts two parameters: the ground truth (`y_true`) and the model predictions (`y_pred`).  
If we want to include a hyperparameter that we can tune, then we can define a wrapper function that accepts this hyperparameter.

In [3]:
# wrapper function that accepts the hyperparameter
def my_huber_loss_with_threshold(threshold):
  
    # function that accepts the ground truth and predictions
    def my_huber_loss(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) <= threshold
        small_error_loss = tf.square(error) / 2
        big_error_loss = threshold * (tf.abs(error) - (0.5 * threshold))
        
        return tf.where(is_small_error, small_error_loss, big_error_loss) 

    # return the inner function tuned by the hyperparameter
    return my_huber_loss

In [8]:
model = Sequential([Dense(1, input_shape=[1])])
model.compile(optimizer='sgd', loss=my_huber_loss_with_threshold(1.42))
model.fit(xs, ys, epochs=500, verbose=0)
model.predict([10])

1/1 [==============================] - 0s 48ms/step


array([[18.818563]], dtype=float32)

## Implement Custom loss as a class

In [16]:
class MyHuberLoss(Loss):

  def __init__(self, threshold=1):
    super().__init__()
    self.threshold = threshold

  
  def call(self, y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) <= self.threshold
    small_error_loss = tf.square(error) / 2
    big_error_loss = self.threshold * (tf.abs(error) - (0.5 * self.threshold))
    return tf.where(is_small_error, small_error_loss, big_error_loss)    

In [17]:
model = Sequential([Dense(1, input_shape=[1])])
custom_loss = MyHuberLoss(0.45)
model.compile(optimizer='sgd', loss=custom_loss)
model.fit(xs, ys, epochs=500, verbose=0)
model.predict([10])

1/1 [==============================] - 0s 50ms/step


array([[18.136822]], dtype=float32)